# Main Cleaning Process and Exploration Analysis
**This notebook serves as the main data cleaning and processing module**

In this notebook , I will 
1. Combine all the diagnosis records from different sections.
2. Implement the chronological filtering to determine the earliest date of pre-diabetes and diabetes. 
3. Merge with demographic information and calculate the age of each time point(pre, diab, death)
4. Exclude some invalid records and label the patients.
5. Do the exploration analysis where I will give out the basic statistic and plot various of charts to illustrate the basic demographic outlines.

For clarity: we name all the pre-diabetes patients with prefix pre,diabetes patients with prefix diab and the patients progressed from pre-diabetes to diabetes with prefix pre2Diab.

We also label the abnormal patients as follow:
- Pre-diabetes Patient (Only have pre-diabetes): 0
- Pre-diabetes to Diabetes paitient (the patients who has sign of pre-diabetes BEFORE they are diagosed with diabetes): 1
- Diabetes (Only have diabetes or have earlier confirmed as diabetes than pre-diabetes): 2

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
import datetime
import random
import time
import shap
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import seaborn as sns
from typing import *
%autoreload 2

In [ ]:
COLORS = [
    "#F27970",
    "#BB9727",
    "#54B345",
    "#32B897",
    "#05B9E2",
    "#8983BF",
    "#C76DA2"
]

COLORS
COLOR_MAP = shap.plots.colors.red_blue

# 0 Generate Pseudo Data

In [ ]:
FEATURES = {
    "lab_tests" : ["hba1c", "fasting_glucose", "creatinine", "ldl_c", "hdl_c", "triglyceride", "potassium"],
    "demographic" : ["age", "sex"]
}

## 1. Combine All The Records From Different  Sections

We read all the table we want from disk:
- Lab Test result
- Diagnosis result (Diabetes Only)
- Family Medicine
- DMCS
- Medication (drug)

In [ ]:
file_path = r'../tables/output'

# read the diag result from each tables
lab_diag = pd.read_csv(r'../tables/output/first_diag_lab.csv', index_col=0)
dx_diag = pd.read_csv(r'../tables/output/first_diag_dx.csv', index_col=0)
fm_diag = pd.read_csv(r'../tables/output/first_diag_fm.csv', index_col=0)
dmcs_diag = pd.read_csv(r'../tables/output/first_diag_dmcs.csv', index_col=0)
drug_diag = pd.read_csv(r'../tables/output/first_diag_drug.csv', index_col=0)

In [ ]:
# combine the diagnosis together
combine = pd.concat([lab_diag, dx_diag, fm_diag, dmcs_diag, drug_diag]).reset_index(drop=True)
combine

In [ ]:
# convert the data type
combine["diff_hour"] = combine["diff_hour"].astype('int')

## 2. Implement the Chronological Filtering

Now, we got the patient diagnosis of pre-diabetes and diabetes for every patient. we need to seperate them into three groups:
1. group 0: no pre-diabetes or diabetes records were earlier than pre-diabetes
2. group 1: only pre-diabetes
3. group 2: pre-diabetes to diabetes

In [ ]:
start = time.time()

def chronoFilter(df: pd.DataFrame) -> pd.DataFrame:
    '''
    callable object than can apply splittng rules that define each patient labels on each patient records. The splitting 
    rules are defined as follow:
    - Pre-diabetes Patient (Only have pre-diabetes): 0
    - Pre-diabetes to Diabetes paitient (the patients who has sign of pre-diabetes BEFORE they are diagosed with diabetes): 1
    - Diabetes (Only have diabetes or have earlier confirmed as diabetes than pre-diabetes): 2
    
    Args:
        df: grouped data frame that is waiting to be aggregated
    Return:
        aggregated data frame
    
    '''
    
    dim = df.shape[0]
    assert dim <= 2 # make sure there are only earliest record(s) in this dataframe
    
    # one record only situation
    if dim == 1: 
        if df.iloc[0]["diab_type"] == "pre":
            return pd.DataFrame({
                "pseudo_patient_key":[df.iloc[0,0]], 
                "pre_dtm": [df.iloc[0,1]],
                "diab_dtm": [np.nan],
                "pre_diff_hour": [df.iloc[0,2]],
                "diab_diff_hour": [np.nan],
                "pre_src": [df.iloc[0,4]],
                "diab_src":[np.nan],
                "label": [0]
            })
        else: #diabetes
            return pd.DataFrame({
            "pseudo_patient_key":[df.iloc[0,0]], 
            "pre_dtm": [np.nan], 
            "diab_dtm": [df.iloc[0,1]],
            "pre_diff_hour": [np.nan],
            "diab_diff_hour": [df.iloc[0,2]],
            "pre_src": [np.nan],
            "diab_src":[df.iloc[0,4]],
            "label": [2]
        })
        
    # two records situation
    else: 
        if df.iloc[0]["diff_hour"] > df.iloc[1]["diff_hour"]: # pre to diabetes
            return pd.DataFrame({
            "pseudo_patient_key":[df.iloc[0,0]], 
            "pre_dtm": [df.iloc[1,1]], 
            "diab_dtm": [df.iloc[0,1]],
            "pre_diff_hour": [df.iloc[1,2]],
            "diab_diff_hour": [df.iloc[0,2]],
            "pre_src": [df.iloc[1,4]],
            "diab_src":[df.iloc[0,4]],
            "label": [1]
        })
        else: # diabetes to pre
            return pd.DataFrame({
            "pseudo_patient_key":[df.iloc[0,0]], 
            "pre_dtm": [np.nan], 
            "diab_dtm": [df.iloc[0,1]],
            "pre_diff_hour": [np.nan],
            "diab_diff_hour": [df.iloc[0,2]],
            "pre_src": [np.nan],
            "diab_src":[df.iloc[0,4]],
            "label": [2]
        })

        
#######################################################################################
# get the row number 
rnk = tools.row_number(combine, "pseudo_patient_key", "diab_type", sort_key="diff_hour")
# get the earliest records for each patient each diab type
el_rec = combine[rnk == 1].sort_values(["pseudo_patient_key", "diab_type"])
# apply the rules on the dataframe
group_patient = el_rec.groupby(by="pseudo_patient_key").apply(chronoFilter)
# write the data to disk
group_patient.to_csv("../tables/output/group_patient.csv")

min = (time.time() - start) / 60
print("runtime: {:.4f} minutes".format(min))

## 3. Merge with demographic information and test results.

### 3.1 Demographic Infomation

In [ ]:
# read the file of patient demographic information
patient_info = tools.fileReader(r"../DATAFILE", 'patient_data')
# read the grouped patients
group_patient = pd.read_csv(r"../tables/output/group_patient.csv", index_col=0)

In [ ]:
left = group_patient
right = patient_info[["pseudo_patient_key", "dob_Y", "sex", "death_date_Y", "diff_in_hour_death_date"]]
diab_patients_info = pd.merge(left=left, right=right, how='left', on='pseudo_patient_key')

Get the age of each date time.

In [ ]:
def map_age(df, fields, dob="dob_Y"):
    '''
    map the date time fieds into age inplace
    Args:
        field: date fields
        dob: date of birth
    '''
        
    age_fields = list(map(lambda x : x.split(r'_')[0] + "_age", fields))
    for af, f in zip(age_fields, fields):
        df[af] = (pd.to_datetime(df[f]) - pd.to_datetime(df[dob])).apply(lambda x : x / np.timedelta64(1, "Y"))
        
######################################################################################################################  

# replace null value to np.nan
diab_patient_age = diab_patients_info.replace(r'""', np.nan)
# map the date time fields into age inplace
map_age(diab_patient_age, ["pre_dtm", "diab_dtm", "death_date_Y"])

# Generate Pseudo patient data

In [ ]:
def generate_random_date(start_year=2003, end_year=2019):
    # Generate a random date within the specified range
    start_date = datetime.date(start_year, 1, 1)
    end_date = datetime.date(end_year, 12, 31)
    days_between = (end_date - start_date).days
    random_days = random.randint(0, days_between)
    random_date = start_date + datetime.timedelta(days=random_days)
    return random_date.strftime('%Y-%m-%d')

In [ ]:
N = 10000
data = {}
for lt in FEATURES["lab_tests"]:
    low = np.random.randint(0,100)
    data[lt] = [np.random.uniform(low, low+20) if np.random.uniform(0, 1) > np.random.uniform(0, 1) * 0.5 else np.nan for _ in range(N)]
    
data["age"] = np.random.uniform(18, 120, N)
data["sex"] = (np.random.uniform(0, 1, N) < 0.45).astype("int")
data["diab_diff_hour"] = np.random.randint(1000,10000, N)
data["diff_in_hour_death_date"] = [np.random.randint(10000, 15000) if np.random.uniform(0, 1) < 0.2 else np.nan for _ in range(N)]
data["pre_diff_hour"] = data["diab_diff_hour"] - np.random.randint(100, 1000)
data["dob_Y"] = np.random.randint(1965, 1996, N).astype("str")
data["label"] = np.random.randint(0, 3, N)
data["pre_dtm"] = [generate_random_date() for _ in range(N)]
data["pre_age"] = map(lambda x : int(x[:4]), data["pre_dtm"])
diab_patient_age = pd.DataFrame(data)

## 4 Exclusion and Labeling

### 4.1 Exclusion
exclusion criterial:
 - Enrolment in last {{TIME_SPEC}} (pre_dtm <= {{CUT_OFF}})
 - Follow up time is less than {{TIME_SPEC}}, we want to exclude the patients who exited the investigation out of death.
 - Diabetes only.
 - Patients younger than 18 (pre_age < 18)

In [ ]:
# convert to year of birth
diab_patient_age["dob_Y"] = diab_patient_age["dob_Y"].apply(lambda x : x[:4]).astype("int")
# compute the progression period in hours
diab_patient_age["prog_pd"] = diab_patient_age["diab_diff_hour"] - diab_patient_age["pre_diff_hour"] 
diab_patient_age["diff_in_hour_death_date"] = diab_patient_age["diff_in_hour_death_date"].astype("float")

In [ ]:
MIN_FT = 30.5 * 24
# exclusion

def show_num():
    print("total: ", diab_patient_excluded.shape[0], 
    "label 0: ", diab_patient_excluded.query("label == 0").shape[0], 
    "label 1: ", diab_patient_excluded.query("label == 1").shape[0]
    )
    
diab_patient_excluded = diab_patient_age.copy() # create a new reference

show_num()
# we first exclude the patients with label 2(diabetes before pre-diabetes)
diab_patient_excluded = diab_patient_excluded.query("~(label == 2)")
show_num()
# we exclude the patients who died before possibly diagnosed with T2DM
diab_patient_excluded = diab_patient_excluded.query("~(label == 0 and diff_in_hour_death_date.notnull())")
show_num()
# we exclude the patients who exit the cohort before possibly diagnosed with T2DM
TIME_SPEC = 10
CUT_OFF = '{year}-12-31'.format(year=2019-TIME_SPEC)
diab_patient_excluded = diab_patient_excluded.query("~(label == 0 and pre_dtm > '{CUT_OFF}')")
show_num()
# Patients younger than 18(pre_age < 18)
diab_patient_excluded = diab_patient_excluded.query("pre_age >= 18")
show_num()

# # Follow-up time less than 1 month i.e. 30.5*24 hours
# diab_patient_age = diab_patient_age.query(f"prog_pd > {MIN_FT} | prog_pd.isnull()", engine='python')

### 4.2 Labeling

In [ ]:
# diab_patient_age = diab_patient_age.assign(prog_pd = diab_patient_age["diab_diff_hour"] - diab_patient_age["pre_diff_hour"])
year_hours = 24 * 365.25
def cls_mapper(prog_pd: float) -> int:
    if prog_pd < 2 * year_hours:
        return 0
    elif prog_pd < 5 * year_hours:
        return 1
    elif prog_pd < 10 * year_hours:
        return 2
    return 3

# uncomment it in the HA
# diab_patient_excluded["cls"] = diab_patient_excluded["prog_pd"].apply(cls_mapper)
diab_patient_excluded["cls"] = np.random.randint(0, 4, diab_patient_excluded.shape[0])

## 5. Explanatory Analysis
I will plot distribution plot 

In [ ]:
# define helper functions
def plot_barchart(s:pd.Series, n_bin:int, bin_width:int=1, sort=False, title=""):
    bin = pd.cut(s, bins=[bin_width * i for i in range(n_bin)])
    out = bin.value_counts(sort=sort)
    ax = out.plot.bar(rot=0, color='b', figsize=(6,4))
    ax.set_xticklabels([bin_width * x for x in range(n_bin)])
    ax.set_title(title)

def plot_boxplot(df, cat, title):
    # boxplot for death age
    fig, axs = plt.subplots(1,len(cat))
    fig.suptitle(title)
    for idx, name in zip(range(len(cat)), cat):
        data = df[df.label == idx]["death_age"]
        axs[idx].boxplot(data, 0, '')
        axs[idx].set_title(name)
    

def show_number(df):
    tools.getNum(df, False)

### 5.1 BoxPlot

In [ ]:
# violin plot
def plot_box(df: pd.DataFrame, features: Sequence[str], cls="cls"):

    # Create the figure and axes object
    fig = plt.figure(figsize=(15, 10))
    n = len(features) // 3 + 1
    for i, feature in enumerate(features):
        df_melted = df[[feature, cls]].melt(id_vars=cls, value_vars=feature,
                        var_name="Measurements", value_name="Value")
        ax = plt.subplot(n, 3, i+1)

        # Creating the violin plot on the axes object
        colors = {}
        for i, color in enumerate(COLORS):
            colors[i] = color
        sns.despine(left=True)
        ax = sns.violinplot(x="Measurements", y="Value", hue="cls", data=df_melted, palette=colors, gap=0.75)
        ax.set_xlabel("")
        ax.set_ylabel("")
        plt.legend().set_visible(False)


features = []
features.extend(FEATURES["lab_tests"])
features.extend(FEATURES["demographic"])

plot_box(diab_patient_excluded, features)
fig.subplots_adjust(hspace=0.4)

In [ ]:
diab_patient_excluded

### 5.2 Correlation Plot

In [ ]:
plt.figure(figsize=(10, 8))
corr_matrix = diab_patient_excluded[features].corr()
# Set up the matplotlib figure
_ = plt.figure(figsize=(10, 8))
# Draw the heatmap with the mask and correct aspect ratio
ax = sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="tab10", cbar_kws={'label': 'Correlation coefficient'})
plt.savefig("correlation_plot.svg", format="svg", dpi=700)

In [ ]:
import pandas as pd
from scipy.stats import kruskal, chi2_contingency
import scipy.stats as stats
# Load the dataset


# Define the features and the group
features = ["hba1c", "fasting_glucose", "creatinine", "ldl_c", "hdl_c", "triglyceride", "potassium", "age", "sex"]

group = 'cls'

statistics = ["missing_rate",
    "pvalue",
    "g0_mean",
    "g1_mean",
    "g2_mean",
    "g3_mean",
    "g0_std",
    "g1_std",
    "g2_std",
    "g3_std",
    "g0_iqr",
    "g1_iqr",
    "g2_iqr",
    "g3_iqr",
]

class StatTape():

    missing_rate = []
    pvalue = []

    g0_mean = []
    g1_mean = []
    g2_mean = []
    g3_mean = []

    g0_std = []
    g1_std = []
    g2_std = []
    g3_std = []

    g0_iqr = []
    g1_iqr = []
    g2_iqr = []
    g3_iqr = []

def stat_append(name: str, value: Any):
    getattr(StatTape, name).append(value)

for feature in features:
    # perform chi square test
    stat_append("missing_rate", diab_patient_excluded[feature].isnull().mean())
    if feature == "sex":
        table = []
        for label in [0,1,2,3]:
            female = diab_patient_excluded.query(f"cls == {label} and sex == 0").count()
            male = diab_patient_excluded.query(f"cls == {label} and sex == 1").count()
            table.append([female, male])
        pvalue = chi2_contingency(table).pvalue
    else:
        groups = diab_patient_excluded.groupby(group)[feature].apply(list)
        pvalue = kruskal(*groups, nan_policy="omit").pvalue

    stat_append("pvalue", pvalue)

    for label in [0, 1, 2, 3]:
        temp = diab_patient_excluded.query(f"cls == {label}")[feature]
        mean = temp.mean()
        std = temp.std()
        iqr = stats.iqr(temp.to_list(), nan_policy="omit")
        stat_append(f"g{label}_mean", mean)
        stat_append(f"g{label}_std", std)
        stat_append(f"g{label}_iqr", iqr)

d = {"features": features}
for s in statistics:
    d[s] = getattr(StatTape, s)
feature_stats = pd.DataFrame(d)

In [ ]:
dataset = diab_patient_excluded.copy() # create new reference

## 5.3 Handling Missingness

I will handle the missingness following steps:
1. exclude the patients without HbA1c, Fasting Glucose.
2. drop the variables that exceeds 30% missingness over the remaining population.
3. check the missingness mechanism, and impute using MICE.
4. mssingness not at random(MNAR) is ruled out from the discussion the given the natural of the clinical input.


In [ ]:
# exclude the patients without HbA1c, Fasting Glucose
dataset = dataset.query("hba1c.notnull() and fasting_glucose.notnull()", engine="python")

# drop the variables that exceeds 25% missingness over the remaining population.
mask = dataset[features].isnull().mean() < 0.25
valid_features = dataset[features].columns[mask].to_list()

In [ ]:
# check the missing mechanism
def hold_out(name, targets):
    res = []
    for target in targets:
        if target != name:
            res.append(target)
    return res

def check_mm(valid_features, df, alpha=0.05):
    '''
    check the missing assumption
    '''
    res = {}
    for target in valid_features:
        remain_feature = hold_out(target, targets)
        ds = df.copy()
        if ds[target].isnull().sum() == 0:
            res[target] = np.nan
            continue 

        dummy = ds[target].isnull().astype(bool)
        table = []
        for label in [0,1,2,3]:
            miss = ds[dummy].query(f"cls == {label}")[target].shape[0]
            non_miss = ds[~dummy].query(f"cls == {label}")[target].shape[0]
            table.append([miss, non_miss])
            
        result = chi2_contingency(table).pvalue
        res[target] = "significant" if result < alpha else "non-significant"
    return res

check_mm(valid_features, dataset)

### Imputation
a rule of thumb for determining the number of
mputations is to use at least the percentage of incomplete cases or more[1].

The imputation should also be implemented within cross-validation[2].

[1] Bodner, T. E. (2008). What improves with increased missing data imputations?. Structural equation modeling: a multidisciplinary journal, 15(4), 651-675.

[2] https://missingdatasolutions.rbind.io/2021/02/mi-cross-validation/

In [ ]:
valid_features

In [ ]:
label_and_features = valid_features
label_and_features.append("cls")
dataset[label_and_features].to_csv("dataset.csv", index=False)

In [ ]:




## we will IterativeImputer
